In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [4]:
@everywhere using Base: @kwdef
@everywhere using Distributed, DistributedStreams, DistributedArrays

## Start Sentinel

In [5]:
control_messages, control_responses, control = launch_sentinel(;
    workers=[2], verbose=true, buffer_size=1024, timeout=1
)

(RemoteChannel{Channel{ControlMessage}}(1, 1, 19), RemoteChannel{Channel{ControlMessage}}(1, 1, 20), DistributedStreams.RemoteWorkerControl(@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}[(worker = 2, flag = Base.RefValue{Bool}(false))], Any[Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2, 126114975044800, 81604378642)), nothing)]))

In [6]:
sentinel_status_t = @async fetch(only(control.status))
sentinel_running() = ! istaskdone(sentinel_status_t)

sentinel_running (generic function with 1 method)

In [7]:
sentinel_running()

true

## Test Managed Worker

In [8]:
ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

@everywhere function test_fn(i)
    println("hi there, I'm running on pid=$(myid())")
    if i == 1
        error("bad input")
    end
    return i+1
end

In [9]:
m_f = ControlMessage(
    message_type=DistributedStreams.start,
    target=3,
    payload=FunctionPayload(
        f=test_fn,
        in=ch_in,
        out=ch_out
    )
)

ControlMessage(DistributedStreams.start, 3, FunctionPayload(test_fn, RemoteChannel{Channel{Int64}}(1, 1, 25), RemoteChannel{Channel{Int64}}(1, 1, 26)))

In [10]:
put!(control_messages, m_f)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 3
      From worker 2:	Worker started for: 3


In [11]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 3, nothing)

### Test worker's ability to run tasks

In [12]:
put!(ch_in, 10)

RemoteChannel{Channel{Int64}}(1, 1, 25)

      From worker 3:	hi there, I'm running on pid=3


In [13]:
collect!(ch_out)

1-element Vector{Int64}:
 11

## Simumulate sudden termination of worker process

In [14]:
rmprocs(3)

Task (done) @0x000072b345bec010

      From worker 2:	Worker 3 died
      From worker 2:	Worker 3 completed


In [15]:
collect!(control_responses)

2-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.failed, 3, nothing)
 ControlMessage(DistributedStreams.completed, 3, ReturnPayload(ProcessExitedException(3)))

In [16]:
workers()

1-element Vector{Int64}:
 2

In [17]:
sentinel_running()

true

## Test adding additional worker processes

In [18]:
addprocs(1)
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [19]:
workers()

2-element Vector{Int64}:
 2
 4

### Test Incorrect Payload

In [20]:
m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    payload=nothing
)

ControlMessage(DistributedStreams.start, 4, nothing)

In [21]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4


In [22]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.completed, 4, ReturnPayload(ArgumentError("Start request does not contain function")))

### Test correct start payload

In [23]:
m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    payload=m_f.payload
)

ControlMessage(DistributedStreams.start, 4, FunctionPayload(test_fn, RemoteChannel{Channel{Int64}}(1, 1, 25), RemoteChannel{Channel{Int64}}(1, 1, 26)))

In [24]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4
      From worker 2:	Worker started for: 4
      From worker 2:	Worker 4 completed


## Test expected failure due to undefined function

In [25]:
collect!(control_responses)

2-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 4, nothing)
 ControlMessage(DistributedStreams.completed, 4, ReturnPayload(RemoteException(4, CapturedException(UndefVarError(Symbol("#test_fn")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814, 1), (#5 at Serialization.jl:973, 1), (ntupleany at ntuple.jl:43, 1), (deserialize_tuple at Serialization.jl:973, 1), (handle_deserialize at Serialization.jl:857, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:892 [inlined], 1), (invokelatest at essentials.jl:889 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_streams at process_messages.jl:133, 1), (#103 at process_messages.jl:121, 1)]))))

### Send function and try again

In [26]:
sendfunc(m_f.payload.f, 4)

In [27]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4
      From worker 2:	Worker started for: 4


In [28]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 4, nothing)

### Worker works normally now

In [30]:
put!(ch_in, 2)

RemoteChannel{Channel{Int64}}(1, 1, 25)

      From worker 4:	hi there, I'm running on pid=4


In [31]:
collect!(ch_out)

1-element Vector{Int64}:
 3

## Test deliberate failure of worker

In [39]:
put!(ch_in, 1)

RemoteChannel{Channel{Int64}}(1, 1, 25)

      From worker 4:	hi there, I'm running on pid=4
      From worker 2:	Worker 4 completed


In [40]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.completed, 4, ReturnPayload(RemoteException(4, CapturedException(ErrorException("bad input"), Any[(error at error.jl:35, 1), (test_fn at In[8]:7, 1), (macro expansion at task.jl:479 [inlined], 1), (remote_worker at DistributedStreams.jl:156, 1), (#invokelatest#2 at essentials.jl:892, 1), (invokelatest at essentials.jl:889, 1), (#107 at process_messages.jl:283, 1), (run_work_thunk at process_messages.jl:70, 1), (run_work_thunk at process_messages.jl:79, 1), (#100 at process_messages.jl:88, 1)]))))

## Test Start Stop

In [33]:
m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    payload=m_f.payload
)

put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4


In [34]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 4, nothing)

In [29]:
m = ControlMessage(
    message_type=DistributedStreams.stop,
    target=4,
    payload=nothing
)

put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Stop instruction for 4


In [32]:
collect!(control_responses)

ControlMessage[]

In [34]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.completed, 4, ReturnPayload(ProcessExitedException(4)))

In [33]:
rmprocs(4)

Task (done) @0x00007af5b2b61aa0

      From worker 2:	Worker 4 completed


In [35]:
workers()

1-element Vector{Int64}:
 2

## Test Graceful Shutdown

In [36]:
stop_workers!(control; workers=[2])

      From worker 2:	Sentinel on 2 is shutting down


In [37]:
rmprocs()

Task (done) @0x00007af5aff7d460

In [38]:
sentinel_running()

false